In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_columns', None)

### Užkeliame duomenis

In [2]:
loans = pd.read_excel('duomenys.xlsx')
defaulted_loans = pd.read_excel('duomenys.xlsx')#, sheet_name='Lapas1')

In [3]:
pd.DataFrame(loans['Borrower ID'].value_counts()).reset_index()['Borrower ID'].value_counts()

1     3682
2      963
3      293
4      120
5       53
6       14
8        9
7        7
11       4
9        4
10       2
13       1
12       1
Name: Borrower ID, dtype: int64

### Sukuriame papildomą kintamąjį - paskolų skaičių.

In [4]:
loans['loan_number'] = loans.groupby(['Borrower ID']).cumcount()+1

In [5]:
loans.head()

,Unnamed: 0,Borrower ID,Credit score,Loan amount,Interest rate,Loan term,default,Purpose,Monthly payment,Birth year,Gender,City,Marital status,Children,Residential status,Education,Months at current employer,DTI,Family income,Family liabilities,Borrower income,Borrower liabilities,loan_number
0,153,U-00002597,B,1640,23.49,33,0,kita,70.96,1991,vyras,NaN,nevedęs/netekėjusi,0,nuosavas,aukštasis,62,35,1610.5197,0.0,1610.5197,0.0,1
1,227,U-00003079,A,6000,22.53,66,0,namų remontui,161.36,1981,moteris,Vilnius,išsiskyręs(-usi),2,nuosavas,pagrindinis,4,18,1261.5074,0.0,1261.5074,0.0,1
2,1030,U-00007926,B,3080,21.05,48,1,paskolų refinansavimui,105.26,1964,moteris,Panevėžys,susituokęs(-usi),0,nuosavas,profesinis,71,21,915.7617,0.0,915.7617,0.0,1
3,1062,U-00008093,D,2100,24.95,45,1,namų remontui,77.24,1964,moteris,Druskininkai,susituokęs(-usi),0,nuosavas,profesinis,24,30,711.6840,0.0,711.6840,0.0,1
4,1107,U-00008375,B,4200,22.84,48,1,paskolų refinansavimui,147.82,1957,moteris,Kuršėnai,susituokęs(-usi),0,nuosavas,profesinis,204,31,1166.9114,0.0,1166.9114,0.0,1


### Pasirenku kintamuosius modelio kūrimui, sukuriami papildomi dummy kintamieji

In [6]:
small_loans = loans[['Loan amount', 'loan_number', 'Family income', 'Family liabilities', 'default', 'Loan term', 'Residential status', 'Interest rate', 'Credit score', 'Gender']]
small_loans = pd.get_dummies(small_loans, columns=['Residential status', 'Credit score', 'Gender'])
small_loans


,Loan amount,loan_number,Family income,Family liabilities,default,Loan term,Interest rate,Residential status_gyvena su tėvais,Residential status_kita,Residential status_nuomojamas,Residential status_nuosavas,Credit score_A,Credit score_B,Credit score_C,Credit score_D,Gender_moteris,Gender_vyras
0,1640,1,1610.5197,0.00,0,33,23.49,0,0,0,1,0,1,0,0,0,1
1,6000,1,1261.5074,0.00,0,66,22.53,0,0,0,1,1,0,0,0,1,0
2,3080,1,915.7617,0.00,1,48,21.05,0,0,0,1,0,1,0,0,1,0
3,2100,1,711.6840,0.00,1,45,24.95,0,0,0,1,0,0,0,1,1,0
4,4200,1,1166.9114,0.00,1,48,22.84,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7557,4350,1,1231.2600,220.59,0,24,14.00,0,0,0,1,0,1,0,0,0,1
7558,2175,1,431.4300,0.00,0,24,15.00,0,0,0,1,0,1,0,0,1,0
7559,4200,2,580.7200,0.00,0,60,25.00,0,0,1,0,0,0,0,1,1,0
7560,3745,1,1148.8800,241.95,0,54,23.00,1,0,0,0,0,0,0,1,0,1


### Padaliname duomenis į priklausomą ir nepriklausomus kintamuosius.

In [7]:
X = small_loans.drop(['default'], axis = 1)
y = loans.default.values

### Padaliname duomenis į train ir test grupes.

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
X_train.shape, X_test.shape

((5671, 16), (1891, 16))

### Sukuriame XGBOOST klasifikavimo modelį.

In [10]:
from xgboost import XGBClassifier
xgbc = XGBClassifier(max_depth = 14, n_estimators=500, learning_rate = 2)
xgbc

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=2, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=14,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)

### Į modelį įdedame duomenis modelio apmokymui.

In [11]:
from sklearn.model_selection import cross_val_score
xgbc.fit(X_train, y_train)

scores = cross_val_score(xgbc, X_test, y_test, cv=5)
scores

array([0.8944591 , 0.86243386, 0.85449735, 0.88624339, 0.87037037])

### Atliekame prognozes su modeliu

In [12]:
y_pred = xgbc.predict(X_test)

### Prognozavimo rezultatus peržiūrime Confusion matrix.

In [13]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, xgbc.predict(X_test))

array([[1638,  120],
       [ 112,   21]], dtype=int64)

### Peržiūrime modelio metrikas.

In [14]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y_test, y_pred)
print('Roc_Auc_Score: %f' % roc_auc_score)

Accuracy: 0.877314
Precision: 0.148936
Recall: 0.157895
F1 score: 0.153285
Roc_Auc_Score: 0.544818


In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import requests
import json

# Saving model to disk
pickle.dump(xgbc, open('model.pkl','wb'))
# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
#print(model.predict([[1.8]]))



In [20]:
extra_small_loans = loans[['Loan amount', 'Family income', 'default', 'Loan term']]
X2 = extra_small_loans.drop(['default'], axis = 1)
y2 = loans.default.values

from sklearn.model_selection import train_test_split
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2)

from xgboost import XGBClassifier
xgbc2 = XGBClassifier(max_depth = 14, n_estimators=500, learning_rate = 2)
xgbc2

from sklearn.model_selection import cross_val_score
xgbc2.fit(X2_train, y2_train)

scores = cross_val_score(xgbc2, X2_test, y2_test, cv=5)
y2_pred = xgbc2.predict(X2_test)

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y2_test, y2_pred)
print('Accuracy: %f' % accuracy)
from sklearn.metrics import precision_score
precision = precision_score(y2_test, y2_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
from sklearn.metrics import recall_score
recall = recall_score(y2_test, y2_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
from sklearn.metrics import f1_score
f1 = f1_score(y2_test, y2_pred)
print('F1 score: %f' % f1)
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y2_test, y2_pred)
print('Roc_Auc_Score: %f' % roc_auc_score)

Accuracy: 0.875198
Precision: 0.150000
Recall: 0.152174
F1 score: 0.151079
Roc_Auc_Score: 0.542145


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import requests
import json

# Saving model to disk
pickle.dump(xgbc2, open('model2.pkl','wb'))
# Loading model to compare the results
model = pickle.load(open('model2.pkl','rb'))
#print(model.predict([[1.8]]))


In [38]:
columns = ['Loan amount', 'Family income', 'Loan term']
values = pd.DataFrame([[645, 620.69, 12]], columns=columns)
model.predict(values).tolist()

[1]

In [37]:
X2_train[24:]

,Loan amount,Family income,Loan term
5993,645,620.69,12
7557,4350,1231.26,24
3393,4020,648.79,60
102,8000,1079.19,60
133,4395,520.51,36
...,...,...,...
191,4420,616.91,30
5434,3260,675.26,36
5504,1675,1126.87,12
4481,1500,523.69,36


In [35]:
for idx, value in enumerate(a):
    if value == 1

24
26
105
106
140
161
166
171
193
214
221
240
250
263
278
291
294
306
316
319
342
363
374
380
385
396
410
411
420
431
465
485
489
521
549
584
609
612
645
679
701
728
739
746
769
797
800
804
810
815
841
843
848
861
879
890
895
929
938
969
970
999
1000
1002
1011
1043
1052
1071
1094
1120
1138
1140
1151
1153
1162
1234
1239
1240
1272
1313
1324
1338
1353
1356
1364
1386
1389
1406
1473
1475
1481
1524
1529
1536
1543
1555
1564
1637
1658
1659
1670
1678
1684
1709
1717
1731
1805
1857
1862
1869
1870
1876
1881
1883
1884
1911
1923
1937
1957
1962
1992
1993
1996
1999
2001
2005
2009
2021
2025
2038
2059
2060
2071
2096
2114
2116
2125
2128
2135
2140
2146
2159
2160
2201
2224
2244
2276
2309
2361
2363
2365
2379
2389
2449
2453
2492
2501
2505
2507
2524
2530
2533
2540
2541
2567
2583
2591
2610
2621
2635
2637
2642
2644
2646
2648
2665
2668
2674
2696
2704
2708
2712
2733
2736
2748
2760
2761
2791
2793
2796
2800
2814
2822
2825
2827
2830
2834
2859
2876
2913
2914
2926
2932
2939
2951
2987
3003
3044
3055
3067
3093
3101
3120